In [6]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd

def train_model(X, y, h=1, m_=1):
    n, m = X.shape
    # --- Initialisation du modèle ---
    mdl = Model(name="CreditClassification")
    
    # Variables
    w = mdl.continuous_var_list(m, name="w")  # Coefficients
    k = mdl.continuous_var(name="k")          # Constante
    E = mdl.continuous_var_list(n, lb=0, name="E")  # Écarts externes
    I = mdl.continuous_var_list(n, lb=0, name="I")  # Écarts internes
    
    # Variables auxiliaires pour la valeur absolue
    w_abs = mdl.continuous_var_list(m, lb=0, name="w_abs")  # |w|
    k_abs = mdl.continuous_var(lb=0, name="k_abs")          # |k|
    
    # --- Fonction objectif ---
    objective = mdl.sum(h * E[i] - m_ * I[i] for i in range(n))
    mdl.minimize(objective)
    
    # --- Contraintes de discrimination ---
    for i in range(n):
        xi = X[i]
        if y[i] == 1:  # G1
            mdl.add_constraint(mdl.sum(w[j] * xi[j] for j in range(m)) + E[i] == k)
        else:          # G2
            mdl.add_constraint(mdl.sum(w[j] * xi[j] for j in range(m)) - E[i] + I[i] == k)
    
    # --- Contraintes pour la valeur absolue ---
    for j in range(m):
        mdl.add_constraint(w_abs[j] >= w[j])
        mdl.add_constraint(w_abs[j] >= -w[j])
    mdl.add_constraint(k_abs >= k)
    mdl.add_constraint(k_abs >= -k)
    
    # --- Contrainte de normalisation ---
    mdl.add_constraint(mdl.sum(w_abs[j] for j in range(m)) + k_abs == 1)
    
    # --- Résolution ---
    solution = mdl.solve(log_output=True)
    
    if solution:
        w_values = [w[j].solution_value for j in range(m)]
        k_value = k.solution_value
        obj_value = solution.objective_value
        return w_values, k_value, obj_value
    return None

# Chargement des données Australian
df_australian = pd.read_csv("/Users/mac/Documents/S4 et S3 de INSEA /devoir_1_MO/Australian.txt", sep=",")
X_australian = df_australian.iloc[:, :-1].values
y_australian = df_australian.iloc[:, -1].values
y_australian = np.where(y_australian == 0, -1, 1)  # Convertir 0 en -1

# Chargement des données German
df_german = pd.read_csv("/Users/mac/Documents/S4 et S3 de INSEA /devoir_1_MO/german.data-numeric.txt", 
                       sep='\s+',  # Use whitespace as separator
                       header=None)  # No header in the file

# Print data information
print("German dataset shape:", df_german.shape)
print("\nFirst few rows of German data:")
print(df_german.head())

# Prepare features and target
X_german = df_german.iloc[:, :-1].values  # All columns except the last one
y_german = df_german.iloc[:, -1].values   # Last column is the target
y_german = np.where(y_german == 2, -1, 1)  # Convert 2 to -1 and 1 stays 1

# Test sur les données Australian
print("\n=== Résultats sur données Australian ===")
result_australian = train_model(X_australian, y_australian)
if result_australian:
    w, k, obj = result_australian
    print("w =", w)
    print("k =", k)
    print("Valeur objectif =", obj)

# Test sur les données German
print("\n=== Résultats sur données German ===")
result_german = train_model(X_german, y_german)
if result_german:
    w, k, obj = result_german
    print("w =", w)
    print("k =", k)
    print("Valeur objectif =", obj)

German dataset shape: (1000, 25)

First few rows of German data:
   0   1   2   3   4   5   6   7   8   9   ...  15  16  17  18  19  20  21  \
0   1   6   4  12   5   5   3   4   1  67  ...   0   0   1   0   0   1   0   
1   2  48   2  60   1   3   2   2   1  22  ...   0   0   1   0   0   1   0   
2   4  12   4  21   1   4   3   3   1  49  ...   0   0   1   0   0   1   0   
3   1  42   2  79   1   4   3   4   2  45  ...   0   0   0   0   0   0   0   
4   1  24   3  49   1   3   3   4   4  53  ...   1   0   1   0   0   0   0   

   22  23  24  
0   0   1   1  
1   0   1   2  
2   1   0   1  
3   0   1   1  
4   0   1   2  

[5 rows x 25 columns]

=== Résultats sur données Australian ===
Checking license ...
Checking license ...
cpxchecklic: Exiting with code 1

Child process exited with code 1
No license found. [0.02 s]
CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.
cpxchecklic: Exiting with code 1

Child process exited with cod

DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 1406 vars, 718 consts, CPLEX code=1016